In [ ]:
from polars import DataFrame, col
from polars import Float64, Int64

In [ ]:
from collections import namedtuple
Tx = namedtuple("Tx", ("ts", "price", "qty"))

txs = [
    Tx(1, 102, 2),
    Tx(3, 105, 5),
    Tx(4, 105.5, -6),
    Tx(5, 106.5, 3),
    Tx(7, 108, 1),
]

In [ ]:
def condsolidate_txs(
    txs: list,
    *,
    period_start_ts:float,
    period_start_price:float,
) -> DataFrame:
    TXs = DataFrame(txs)
    TXsb = TXs.filter(col("ts") <= period_start_ts)
    TXsa = TXs.filter(col("ts") > period_start_ts)

    TXsb = TXsb.select(
        col("ts").replace_strict({}, default=period_start_ts).cast(Int64),
        col("price").replace_strict({}, default=period_start_price).cast(Float64),
        col("qty").sum(),
    ).tail(1)

    TXs = TXsb.extend(TXsa)
    return TXs


def calc_returns(
    txs: list, *, period_start_ts:int, period_start_price:float, market_price:float,
) -> dict:
    TXs = condsolidate_txs(
        txs,
        period_start_ts=period_start_ts,
        period_start_price=period_start_price,
    )
    TXs = TXs.with_columns(
        ts_start=col("ts"),
        ts_end=col("ts").shift(
            -1
        ),  # fill_null with current_timestamp or ts of market_price
        price_start=col("price"),
        price_end=col("price").shift(-1).fill_null(market_price),
        running_holding=col("qty").rolling_sum(len(TXs), min_samples=1),
        cost=col("qty") * col("price"),
    ).with_columns(
        holding_before=col("running_holding").shift(1).fill_null(0),
        growth_factor=col("price_end") / col("price_start"),
        running_value=col("running_holding") * col("price_end"),
        running_cost=col("cost").rolling_sum(len(TXs), min_samples=1),
    )
    twr = TXs.select(col("growth_factor").alias("twr")).product() - 1
    val = TXs.select(col("running_value").alias("final_value")).tail(1)
    cost = TXs.select(col("cost").sum().alias("final_cost"))
    start_holding = TXs.head(1).select(
        (col("holding_before")*col("price_start")).alias("start_value")
    )  # <-- this is probably wrong

    twr = twr.to_dicts()[0]
    val = val.to_dicts()[0]
    cost = cost.to_dicts()[0]
    start_holding = start_holding.to_dicts()[0]

    return((twr | val | cost | start_holding))




In [208]:
market_price = 110
period_start_ts = 0
period_start_price = 101

res = calc_returns(txs, period_start_price=period_start_price, period_start_ts=period_start_ts,
                   market_price=market_price,)

twr, final_value, final_cost, start_value = res.values()
dollar_return = final_value - start_value - final_cost
mwr = dollar_return / final_cost


print(f"""
    {twr=}
    {mwr=}
    {start_value=}
    {final_value=}
    {final_cost=}
    {dollar_return=}
""")


    twr=0.07843137254901977
    mwr=0.050620821394460364
    start_value=0.0
    final_value=550.0
    final_cost=523.5
    dollar_return=26.5

